In [3]:
# Waifu Diffusion
#!wget https://thisanimedoesnotexist.ai/downloads/wd-v1-2-full-ema.ckpt

#Stable Diffusion
#!wget https://r2-public-worker.drysys.workers.dev/sd-v1-4-full-ema.ckpt

#Poke Diffusion
!wget https://sd-finetune.vonk.workers.dev/pokediffusion_ckpts/pokediffusion_epoch_10_pruned.ckpt

!git clone https://github.com/CompVis/stable-diffusion.git
%cd stable-diffusion
!wget https://raw.githubusercontent.com/justinpinkney/stable-diffusion/main/requirements.txt
!pip install -r requirements.txt
!pip install --upgrade pytorch-lightning
!apt-get update -y && apt-get install libgl1 -y && apt-get install libglib2.0-0 -y

--2022-09-18 19:48:07--  https://sd-finetune.vonk.workers.dev/pokediffusion_ckpts/pokediffusion_epoch_10_pruned.ckpt
Resolving sd-finetune.vonk.workers.dev (sd-finetune.vonk.workers.dev)... 172.67.194.133, 104.21.92.135, 2606:4700:3036::6815:5c87, ...
Connecting to sd-finetune.vonk.workers.dev (sd-finetune.vonk.workers.dev)|172.67.194.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3852166575 (3.6G)
Saving to: ‘pokediffusion_epoch_10_pruned.ckpt’

pokediffusion_epoch 100%[===================>]   3.59G  38.8MB/s    in 98s     

2022-09-18 19:49:47 (37.3 MB/s) - ‘pokediffusion_epoch_10_pruned.ckpt’ saved [3852166575/3852166575]

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 313, done.
remote: Total 313 (delta 0), reused 0 (delta 0), pack-reused 313
Receiving objects: 100% (313/313), 42.62 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/workspace/stable-diffusion/stable-diffusion
--2022-09-18 19:49:51--  https://raw.githubu

In [ ]:
# !! Restart your notebook here !!

In [1]:
%cd stable-diffusion
# Waifu Diffusion
#ckpt_file = "wd-v1-2-full-ema.ckpt"

#Stable Diffusion
#ckpt_file = "sd-v1-4-full-ema.ckpt"

#Poke Diffusion
ckpt_file = "pokediffusion_epoch_10_pruned.ckpt"

/workspace/stable-diffusion


In [2]:
from io import BytesIO
import os
from contextlib import nullcontext

import fire
import numpy as np
import torch
from einops import rearrange
from omegaconf import OmegaConf
from PIL import Image
from torch import autocast
from torchvision import transforms
import requests

from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from ldm.util import instantiate_from_config
from pytorch_lightning import seed_everything

In [3]:
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

In [4]:
config = OmegaConf.load("configs/stable-diffusion/v1-inference.yaml")
model = load_model_from_config(config, f"../{ckpt_file}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

sampler = DDIMSampler(model)

sample_path = "outs"
os.makedirs(sample_path, exist_ok=True)

start_code = None

Loading model from ../pokediffusion_epoch_10_pruned.ckpt
Global Step: 472080
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Downloading: "https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty_with_aug/checkpoint_liberty_with_aug.pth" to /root/.cache/torch/hub/checkpoints/checkpoint_liberty_with_aug.pth


  0%|          | 0.00/5.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.7.layer_norm2.weight', 'vision_model.encoder.layers.4.layer_norm1.weight', 'vision_model.encoder.layers.15.self_attn.q_proj.weight', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.23.mlp.fc2.bias', 'vision_model.encoder.layers.20.mlp.fc1.bias', 'vision_model.encoder.layers.16.self_attn.out_proj.bias', 'vision_model.encoder.layers.23.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.self_attn.out_proj.bias', 'vision_model.encoder.layers.23.layer_norm1.weight', 'vision_model.encoder.layers.22.layer_norm1.weight', 'vision_model.encoder.layers.1.self_attn.out_proj.bias', 'vision_model.encoder.layers.13.self_attn.out_proj.weight', 'vision_model.encoder.layers.10.self_attn.k_proj.weight', 'vision_model.encoder.layers.18.self_attn.v_proj.weight', 'vision_model.encoder.layers.15.self_attn.out_proj.bias', 

In [5]:
import random

In [7]:
import time

In [ ]:
seeds = [random.randint(1, 2000000) for x in range(8)]
for seed in seeds:
    batch_size = 1

    prompt = "official art of a pokemon Abomasnow, Ice, Grass"
    scale = 7.5
    C = 4
    H = 512
    W = 512
    f = 8
    data = [batch_size * [prompt]]

    seed_everything(seed)

    precision_scope = autocast
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                all_samples = list()
                for n in range(1):
                    for prompts in data:
                        uc = None
                        if scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)
                        shape = [C, H // f, W // f]
                        samples_ddim, _ = sampler.sample(S=50,
                                                         conditioning=c,
                                                         batch_size=1,
                                                         shape=shape,
                                                         verbose=False,
                                                         unconditional_guidance_scale=scale,
                                                         unconditional_conditioning=uc,
                                                         eta=0.0,
                                                         x_T=start_code)

                        x_samples_ddim = model.decode_first_stage(samples_ddim)
                        x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                        x_samples_ddim = x_samples_ddim.cpu().permute(0, 2, 3, 1).numpy()

                        x_checked_image_torch = torch.from_numpy(x_samples_ddim).permute(0, 3, 1, 2)

                        for x_sample in x_checked_image_torch:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            img = Image.fromarray(x_sample.astype(np.uint8))
                            img.save(os.path.join(sample_path, f"{prompt[:25]}_{seed}.png"))

                toc = time.time()

Global seed set to 1290026


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:07<00:00,  6.83it/s]
Global seed set to 937160


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:07<00:00,  6.77it/s]
Global seed set to 979059


...

DDIM Sampler: 100%|██████████| 50/50 [00:07<00:00,  6.80it/s]
Global seed set to 871330


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:07<00:00,  6.80it/s]
Global seed set to 1401714


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:07<00:00,  6.73it/s]
